In [66]:
!pip install opencv-contrib-python imutils scikit-image

In [67]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [68]:
!pip install pytz

In [75]:
# import the necessary packages
from skimage.segmentation import clear_border
from pytesseract import image_to_string
import numpy as np
import imutils
import cv2

class PyImageSearchANPR:
	def __init__(self, minAR=4, maxAR=5, debug=False):
		self.minAR = minAR
		self.maxAR = maxAR
		self.debug = debug

	def debug_imshow(self, title, image, waitKey=False):
		if self.debug:
			cv2.imshow(title, image)
			if waitKey:
				cv2.waitKey(0)


	def locate_license_plate_candidates(self, gray, keep=5):

		rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 9))
		blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)
		self.debug_imshow("Blackhat", blackhat)

		squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
		light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
		light = cv2.threshold(light, 0, 255,
			cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
		self.debug_imshow("Light Regions", light)
		gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F,
			dx=1, dy=0, ksize=-1)
		gradX = np.absolute(gradX)
		(minVal, maxVal) = (np.min(gradX), np.max(gradX))
		gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
		gradX = gradX.astype("uint8")
		self.debug_imshow("Scharr", gradX)
		gradX = cv2.GaussianBlur(gradX, (3, 3), 0)
		gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
		thresh = cv2.threshold(gradX, 0, 255,
			cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
		self.debug_imshow("Grad Thresh", thresh)
		thresh = cv2.erode(thresh, None, iterations=2)
		thresh = cv2.dilate(thresh, None, iterations=2)
		self.debug_imshow("Grad Erode/Dilate", thresh)
		thresh = cv2.bitwise_and(thresh, thresh, mask=light)
		thresh = cv2.dilate(thresh, None, iterations=2)
		thresh = cv2.erode(thresh, None, iterations=1)
		self.debug_imshow("Final", thresh, waitKey=True)
		cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
			cv2.CHAIN_APPROX_SIMPLE)
		cnts = imutils.grab_contours(cnts)
		cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:keep]
		return cnts

	def locate_license_plate(self, gray, candidates,
		clearBorder=False):
		lpCnt = None
		roi = None
		for c in candidates:		
			(x, y, w, h) = cv2.boundingRect(c)
			ar = w / float(h)	
			if ar >= self.minAR and ar <= self.maxAR:			
				lpCnt = c
				licensePlate = gray[y:y + h, x:x + w]
				roi = cv2.threshold(licensePlate, 0, 255,
					cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]		
				if clearBorder:
					roi = clear_border(roi)		
				self.debug_imshow("License Plate", licensePlate)
				self.debug_imshow("ROI", roi, waitKey=True)
				break		
		return (roi, lpCnt)

	def build_tesseract_options(self, psm=7):
		alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
		options = "-c tessedit_char_whitelist={}".format(alphanumeric)
		options += " --psm {}".format(psm)
		return options


	def find_and_ocr(self, image, psm=7, clearBorder=False):
		lpText = None
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		candidates = self.locate_license_plate_candidates(gray)
		(lp, lpCnt) = self.locate_license_plate(gray, candidates,
			clearBorder=clearBorder)
		if lp is not None:
			options = self.build_tesseract_options(psm=psm)
			lpText = image_to_string(lp, config=options)
			self.debug_imshow("License Plate", lp)
		return (lpText, lpCnt)

In [76]:
def cleanup_text(text):
	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV
	return "".join([c if ord(c) < 128 else "" for c in text]).strip()

In [77]:
from imutils import paths
anpr = PyImageSearchANPR(debug=True)
imagePaths = sorted(list(paths.list_images('./Database/')))
imagePaths

['./Database/1.jpg',
 './Database/10.jpg',
 './Database/11.jpg',
 './Database/2.jpg',
 './Database/4.jpg',
 './Database/5.jpg',
 './Database/6.jpg',
 './Database/7.jpg',
 './Database/8.jpg',
 './Database/9.jpeg']

In [78]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [79]:
for imagePath in imagePaths:
	# load the input image from disk and resize it
	image = cv2.imread(imagePath,1)
	image = imutils.resize(image, width=600)
	# apply automatic license plate recognition
	(lpText, lpCnt) = anpr.find_and_ocr(image, psm=7,
		clearBorder= 1)
	# only continue if the license plate was successfully OCR'd
	if lpText is not None and lpCnt is not None:
		# fit a rotated bounding box to the license plate contour and
		# draw the bounding box on the license plate
		box = cv2.boxPoints(cv2.minAreaRect(lpCnt))
		box = box.astype("int")
		cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
		# compute a normal (unrotated) bounding box for the license
		# plate and then draw the OCR'd license plate text on the
		# image
		(x, y, w, h) = cv2.boundingRect(lpCnt)
		cv2.putText(image, cleanup_text(lpText), (x, y - 15),
			cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
		# show the output ANPR image
		print("[INFO] {}".format(lpText))
		cv2.imshow("Output ANPR", image)
		cv2.waitKey(0)
		cv2.imwrite("/Results/[RES]{}.jpg".format(imagePath),image)
	else:
		print("this image has no number plate ","{}".format(imagePath))
cv2.destroyAllWindows()

this image has no number plate  ./Database/1.jpg
this image has no number plate  ./Database/10.jpg
this image has no number plate  ./Database/11.jpg
this image has no number plate  ./Database/2.jpg
this image has no number plate  ./Database/4.jpg
this image has no number plate  ./Database/5.jpg
[INFO] 7

this image has no number plate  ./Database/7.jpg
this image has no number plate  ./Database/8.jpg
this image has no number plate  ./Database/9.jpeg


In [74]:
cv2.destroyAllWindows()